# Imports

In [12]:
#general imports
import pandas as pd
import numpy as np

#statsmodels for regression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

#scipy for testing
from scipy import stats

#for visualization
import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

# Weather Data

Import of the weather data

In [13]:
df_weather = pd.read_csv('../data/weather_hourly_philadelphia.csv')

Change type of "date_time" to datetime

In [14]:
df_weather["date_time"] = pd.to_datetime(df_weather["date_time"])

Ordered weather data by date

In [15]:
df_weather = df_weather.sort_values(by=['date_time'])

Deleted weather date which is not needed for Philadelphia 2017 (<2017 or >2017)

In [16]:
start2017 = datetime(2016, 12, 31, 23)
end2017 = datetime(2018, 1, 1)

df_2017weather = df_weather[(df_weather["date_time"] > start2017) & (df_weather["date_time"] < end2017)]
df_2017weather_unique = df_2017weather.drop_duplicates(subset='date_time')
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(10)

<ipython-input-16-6b4f56e891d3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))


,date_time,max_temp,min_temp,precip
Order,,,,
8222,2017-12-31 14:00:00,-9.4,-9.4,0.0
8223,2017-12-31 15:00:00,-8.9,-8.9,0.0
8224,2017-12-31 16:00:00,-8.9,-8.9,0.0
8225,2017-12-31 17:00:00,-8.9,-8.9,0.0
8226,2017-12-31 18:00:00,-8.3,-8.3,0.0
8227,2017-12-31 19:00:00,-8.9,-8.9,0.0
8228,2017-12-31 20:00:00,-8.9,-8.9,0.0
8229,2017-12-31 21:00:00,-9.4,-9.4,0.0
8230,2017-12-31 22:00:00,-10.0,-10.0,0.0


Identify missing data and generate by using average data from former and following hour

In [17]:
for i in df_2017weather_unique.index:
    if i == 8231:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

End


New order by date, because we added new lines

In [18]:
df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(10)

,date_time,max_temp,min_temp,precip
Order,,,,
8749,2017-12-31 14:00:00,-9.4,-9.4,0.0
8750,2017-12-31 15:00:00,-8.9,-8.9,0.0
8751,2017-12-31 16:00:00,-8.9,-8.9,0.0
8752,2017-12-31 17:00:00,-8.9,-8.9,0.0
8753,2017-12-31 18:00:00,-8.3,-8.3,0.0
8754,2017-12-31 19:00:00,-8.9,-8.9,0.0
8755,2017-12-31 20:00:00,-8.9,-8.9,0.0
8756,2017-12-31 21:00:00,-9.4,-9.4,0.0
8757,2017-12-31 22:00:00,-10.0,-10.0,0.0


Running the same code again because one line is missing

In [19]:
for i in df_2017weather_unique.index:
    if i == 8758:
        print("End")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

{'date_time': Timestamp('2017-09-17 05:00:00'), 'max_temp': 21.375, 'min_temp': 21.375, 'precip': 0.0}
End


New order by date, because we added new lines

In [20]:
df_2017weather_unique = df_2017weather_unique.sort_values(by=['date_time'])
df_2017weather_unique['Order'] = np.arange(len(df_2017weather_unique))
df_2017weather_unique = df_2017weather_unique.set_index('Order')

df_2017weather_unique.tail(10)

,date_time,max_temp,min_temp,precip
Order,,,,
8750,2017-12-31 14:00:00,-9.4,-9.4,0.0
8751,2017-12-31 15:00:00,-8.9,-8.9,0.0
8752,2017-12-31 16:00:00,-8.9,-8.9,0.0
8753,2017-12-31 17:00:00,-8.9,-8.9,0.0
8754,2017-12-31 18:00:00,-8.3,-8.3,0.0
8755,2017-12-31 19:00:00,-8.9,-8.9,0.0
8756,2017-12-31 20:00:00,-8.9,-8.9,0.0
8757,2017-12-31 21:00:00,-9.4,-9.4,0.0
8758,2017-12-31 22:00:00,-10.0,-10.0,0.0


In [ ]:
Check if data is complete by checking time difference of following 

In [21]:
for i in df_2017weather_unique.index:
    if i == 8759:
        print("Data complete")
    elif (df_2017weather_unique['date_time'][i] - df_2017weather_unique['date_time'][i+1]) != (df_2017weather_unique['date_time'][1] - df_2017weather_unique['date_time'][2]):
        df_2017weather_support = {'date_time': (df_2017weather_unique['date_time'][i] + pd.DateOffset(hours=1)), 'max_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'min_temp': (df_2017weather_unique['max_temp'][i] + df_2017weather_unique['max_temp'][i+1])/2, 'precip': df_2017weather_unique['precip'][i]}
        print(df_2017weather_support)
        df_2017weather_unique = df_2017weather_unique.append(df_2017weather_support, ignore_index=True)

Data complete
